### Code Hist.

 - CODE : Resample - KIER Energy Usage  
 - DESC  
    &ensp; 1) "M02-01_Data_06_Restore_Energy ACCU.ipynb"로부터 복원된 에너지 적산 사용량을 Resample  
    &ensp; 2) Resample된 데이터를 기반으로 기간별(10MIN, 30MIN, 1H, 12H, 24H, 1W, 2W, 1M) 순시 사용 데이터 산출 및 CSV 추출  

 - DATE  
    &ensp; 2024-08-05 Created  

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [1]:
#region Import_Basic Module
## Basic
import os, sys, warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('./__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
pd.options.display.float_format = '{:.10f}'.format

import math, random

## Datetime
import time, datetime as dt
from datetime import datetime, date, timedelta

## glob
import glob, requests, json
from glob import glob

## 시각화
import matplotlib.pyplot as plt, seaborn as sns
# %matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## Split, 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# Clustering 알고리즘의 성능 평가 측도
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## For Web
import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import tqdm
from tqdm.notebook import tqdm
#endregion Import_Basic Module

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
## Import_Local
from Src_Dev_Common import Data_Datetime as com_date, Data_Preprocessing as com_Prep, KMA_Weather as com_KMA, KECO_AirKor as com_KECO, KASI_Holiday as com_Holi, KIER_Usage_M02 as com_KIER_M02, Data_Analysis as com_Analysis, Data_Visualization as com_Visual, KIER_Usage as com_KIER

### 01-01-02. Config (Directory, Params)

In [4]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [5]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-08-07 09:51:34.849799
2024 / 8 / 7
9 : 51


In [6]:
## Dict_Domain
int_domain = 5

## Domain, ACCU/INST Column
str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
## Directory Root
str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)

## File
str_fileRaw, str_fileRaw_hList = str('KIER_RAW_' + str_domain + '_H_ID_Adopted.csv'), str('KIER_hList_Common.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

5 : GAS
['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_Cleansed', 'KIER_2_BLD', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_GAS_561-1-1_ACCU_01_Raw.csv', 'KIER_GAS_561-1-1_INST_01_10min.csv', 'KIER_GAS_561-1-2_ACCU_01_Raw.csv', 'KIER_GAS_561-1-2_INST_01_10min.csv', 'KIER_GAS_561-1-3_ACCU_01_Raw.csv', 'KIER_GAS_561-1-3_INST_01_10min.csv', 'KIER_GAS_561-1-4_ACCU_01_Raw.csv', 'KIER_GAS_561-1-4_INST_01_10min.csv', 'KIER_GAS_561-10-1_ACCU_01_Raw.csv', 'KIER_GAS_561-10-1_INST_01_10min.csv', 'KIER_GAS_561-10-2_ACCU_01_Raw.csv', 'KIE

## 01-02. Data Load (df_raw)

### 01-02-01. KIER (Energy Usage)

In [7]:
## Combined Data
str_file = 'KIER_' + str_domain + '_ACCU_10MIN_Restored.csv'

df_kier_raw = pd.read_csv(str_dirName_h + str_file, index_col = 0)
# df_kier_raw = com_date.create_col_datetime(df_kier_raw, 'METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE').drop(labels=['None'], axis = 1)
df_kier_raw['METER_DATE'] = pd.to_datetime(df_kier_raw['METER_DATE'])
df_kier_raw = df_kier_raw[['METER_DATE'] + list(df_kier_raw.columns[6:])]
df_kier_raw

,METER_DATE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,GAS_ACCU_FLOW_561-2-1,GAS_ACCU_FLOW_561-2-2,GAS_ACCU_FLOW_561-2-3,GAS_ACCU_FLOW_561-2-4,GAS_ACCU_FLOW_561-3-1,...,GAS_ACCU_FLOW_563-22-4,GAS_ACCU_FLOW_563-22-5,GAS_ACCU_FLOW_563-22-6,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2,None
0,2022-08-26 20:50:00,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,0.6800000000,0.1100000000,1.0900000000,1.1500000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
1,2022-08-26 21:00:00,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,0.6800000000,0.1100000000,1.0900000000,1.1500000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
2,2022-08-26 21:10:00,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,0.6800000000,0.1100000000,1.0900000000,1.1500000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
3,2022-08-26 21:20:00,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,0.6800000000,0.1100000000,1.0900000000,1.1500000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
4,2022-08-26 21:30:00,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,0.6800000000,0.1100000000,1.0900000000,1.1500000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96124,2024-06-24 09:30:00,1.2265582811,4.9229868879,137.6671362594,7.1795856025,41.0541948492,606254.6580253679,4.3510359785,60.3430450399,9350.1639302608,...,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377,0
96125,2024-06-24 09:40:00,1.2265582811,4.9229868879,137.6671362594,7.1799985819,41.0541948492,606254.6580253679,4.3510359785,60.3430450399,9350.1639302608,...,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377,0
96126,2024-06-24 09:50:00,1.2265582811,4.9229868879,137.6671362594,7.1802395457,41.0541948492,606254.6580253679,4.3510359785,60.3430450399,9350.1639302608,...,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377,0
96127,2024-06-24 10:00:00,1.2265582811,4.9229868879,137.6671362594,7.1806346825,41.0541948492,606254.6580253679,4.3510359785,60.3430450399,9350.1639302608,...,81.4748031782,96.1834781938,32.0079067537,4.8163148360,3.4613273905,118.1774391570,127.1390819162,1990.3292276733,3.7353104377,0


## 01-03. Data Resample

In [8]:
## 시간 간격 (10분 / 30분 / 1시간 / 1일 / 1주 / 1달)
list_col_interval = ['10T', '30T', '1H', '12H', '1D', '1W', '2W', '1M']
for str_int in list_col_interval:
    print(str_int)
    df_kier_res = df_kier_raw.resample(str_int, on = 'METER_DATE').last().reset_index()

    if str_int == '10T' : str_int = '10MIN'
    elif str_int == '30T' : str_int = '30MIN'

    str_file = 'KIER_' + str_domain + '_Resample_ACCU_' + str_int + '.csv'
    df_kier_res.to_csv(str_dirName_h + str_file)
df_kier_res

10T
30T
1H
12H
1D
1W
2W
1M


,METER_DATE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,GAS_ACCU_FLOW_561-2-1,GAS_ACCU_FLOW_561-2-2,GAS_ACCU_FLOW_561-2-3,GAS_ACCU_FLOW_561-2-4,GAS_ACCU_FLOW_561-3-1,...,GAS_ACCU_FLOW_563-22-4,GAS_ACCU_FLOW_563-22-5,GAS_ACCU_FLOW_563-22-6,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2,None
0,2022-08-31,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,0.6800000000,0.1100000000,1.0900000000,1.1500000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
1,2022-09-30,0.1513124509,0.2984142492,2.5055268685,0.1961796558,0.1920955712,1.6134859175,0.1420623789,1.4813124509,2.4718491605,...,1.6046185313,2.7742720660,1.8249355418,0.2066235948,0.1911871953,1.8337672850,0.0325823528,2.9513124509,0.1626768962,0
2,2022-10-31,0.7535804139,1.4746275580,7.4624698871,0.8691116875,1.3552611971,5.1489358794,0.7795906433,3.7035804139,10.0892934004,...,6.1968864944,9.9948048345,4.6944662471,1.0941251230,0.9545072195,10.3801538249,14.9786923167,11.5491119282,1.3532129476,0
3,2022-11-30,0.7986373867,1.6359285527,40.2774063015,0.9634392253,39.3482253038,606176.7008290266,0.9221696814,5.3086373867,9222.8519319495,...,9.4396429471,14.3811661689,6.1077144065,1.3373140901,1.1469876669,16.7176520882,22.3907598380,1851.5971141552,1.6922624618,0
4,2022-12-31,0.8208226582,1.6498148894,44.4122368606,1.0321147596,39.4744015115,606180.8464475063,1.1352102905,7.0019427975,9231.5862054530,...,14.2286410851,18.9844108493,7.2614693654,1.5630309381,1.3486505365,21.3401653732,28.6839543988,1860.7771306734,1.8027386943,0
5,2023-01-31,0.8623695692,1.6808580427,49.8407959353,1.2603165466,39.6246801879,606186.6259715462,1.2457399624,9.0474106322,9238.8870339979,...,19.6299234317,22.5684643641,8.3514693654,1.7895212855,1.5015867578,26.6338846605,37.8121503702,1869.3996759211,1.9601075528,0
6,2023-02-28,0.8807624379,1.7055037968,54.0333222097,1.3219498489,39.7061127414,606191.1514475242,1.3175649277,10.6096735999,9245.3955643957,...,23.7053061582,27.6528110450,9.2843065325,1.8673068230,1.6332628927,31.3998530091,43.6274138393,1877.9177023927,2.0343841930,0
7,2023-03-31,0.8807624379,1.7147750447,57.5133695681,1.5112792893,39.7471366313,606195.3390900454,1.3927410738,13.7296735999,9253.0690742667,...,28.0654013321,31.2828110450,10.9143065325,1.8969208676,1.7917297985,37.0343079483,49.7007060459,1886.5142350034,2.1539751114,0
8,2023-04-30,0.8807624379,1.7497725850,61.5884332797,1.7341916926,39.7642390118,606198.9002268070,1.4929344765,17.0796735999,9259.7034681678,...,34.4672998637,36.0228110450,12.5044814601,1.9692249789,1.9301666809,42.7145695762,55.5307935099,1895.0183623954,2.2666709111,0
9,2023-05-31,0.8807624379,1.8139706005,66.3193952185,1.9514157618,39.8082197547,606201.9753942215,1.5481960660,19.4596735999,9266.8823315615,...,38.7630910626,40.0328400305,13.4453620930,2.0360209812,2.0495009006,47.2345695762,61.0627253869,1903.9619125584,2.2974667974,0
